<a href="https://colab.research.google.com/github/zmkarakas/data_science_project_pyspark-plotly/blob/main/data_science_project_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#to view this project, get the kaggle.json file from Kaggle under 'Create new API token' and upload it, then run all the cells.

!pip install  --upgrade plotly
! pip install -q kaggle
from google.colab import files
files.upload()
# upload kaggle.json here

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ziyamertkarakas","key":"19bd6f487f935baef3a94c19f5486ae3"}'}

In [4]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d arjunprasadsarkhel/2021-olympics-in-tokyo
!unzip 2021-olympics-in-tokyo.zip

mkdir: cannot create directory ‘/root/.kaggle’: File exists
2021-olympics-in-tokyo.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  2021-olympics-in-tokyo.zip
replace Athletes.xlsx? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [5]:
!pip install pyspark
#this downloads pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 16.2 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=6cc80c27fcc14efd7aa7e87ad5d11d1840c6d263dded892ec228f66371570f30
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark


In [ ]:
import pyspark
spark = pyspark.sql.SparkSession(pyspark.SparkContext())
spark

In [28]:
import pandas as pd

def excel_to_csv(name):
  pd.read_excel(name + '.xlsx').to_csv(name + '.csv', sep = ',', index = False)

excel_to_csv('Athletes')
excel_to_csv('Coaches')
excel_to_csv('EntriesGender')
excel_to_csv('Medals')
excel_to_csv('Teams')


/usr/local/lib/python3.9/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning:

Workbook contains no default style, apply openpyxl's default



In [29]:
athletes = pd.read_csv('Athletes.csv')
coaches = pd.read_csv('Coaches.csv')
entriesGender = pd.read_csv('EntriesGender.csv')
medals = pd.read_csv('Medals.csv')
teams = pd.read_csv('Teams.csv')
athletes.head()

,Name,NOC,Discipline
0,AALERUD Katrine,Norway,Cycling Road
1,ABAD Nestor,Spain,Artistic Gymnastics
2,ABAGNALE Giovanni,Italy,Rowing
3,ABALDE Alberto,Spain,Basketball
4,ABALDE Tamara,Spain,Basketball


In [30]:
medals.head()

,Rank,Team/NOC,Gold,Silver,Bronze,Total,Rank by Total
0,1,United States of America,39,41,33,113,1
1,2,People's Republic of China,38,32,18,88,2
2,3,Japan,27,14,17,58,5
3,4,Great Britain,22,21,22,65,4
4,5,ROC,20,28,23,71,3


In [31]:
medals.sort_values('Total', inplace=True)
medals.head()

,Rank,Team/NOC,Gold,Silver,Bronze,Total,Rank by Total
92,86,Syrian Arab Republic,0,0,1,1,77
62,63,Bermuda,1,0,0,1,77
76,77,Bahrain,0,1,0,1,77
77,77,Saudi Arabia,0,1,0,1,77
78,77,Lithuania,0,1,0,1,77


In [77]:
import plotly.express as px
!pip install -U kaleido
import plotly.io as pio
pio.renderers.default = "svg"
fig = px.bar(medals, x='Team/NOC', y=['Gold', 'Silver', 'Bronze'], color_discrete_sequence =['Gold', 'Silver', 'Brown'], title='Number of Medals Per Country')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [56]:
coaches.head(5)

,Name,NOC,Discipline,Event
0,ABDELMAGID Wael,Egypt,Football,NaN
1,ABE Junya,Japan,Volleyball,NaN
2,ABE Katsuhiko,Japan,Basketball,NaN
3,ADAMA Cherif,Côte d'Ivoire,Football,NaN
4,AGEBA Yuya,Japan,Volleyball,NaN


In [57]:
spark.read.csv('Coaches.csv', header=True, inferSchema=True).createOrReplaceTempView('Coaches')
spark.sql('Select * from Coaches').show(5)

+---------------+-------------+----------+-----+
|           Name|          NOC|Discipline|Event|
+---------------+-------------+----------+-----+
|ABDELMAGID Wael|        Egypt|  Football| null|
|      ABE Junya|        Japan|Volleyball| null|
|  ABE Katsuhiko|        Japan|Basketball| null|
|   ADAMA Cherif|Côte d'Ivoire|  Football| null|
|     AGEBA Yuya|        Japan|Volleyball| null|
+---------------+-------------+----------+-----+
only showing top 5 rows



In [58]:
pd_coaches_query = spark.sql('Select NOC, Count(NOC) as Count from coaches group by NOC order by NOC').toPandas()
pd_coaches_query.head(7)

,NOC,Count
0,Angola,1
1,Argentina,11
2,Australia,22
3,Austria,3
4,Bahrain,1
5,Belarus,1
6,Belgium,5


In [74]:
fig = px.bar(pd_coaches_query, x= 'NOC', y= 'Count', color='Count', title='Number of Coaches from each country')
fig.show()

ValueError: ignored

In [60]:
spark.read.csv('Athletes.csv', header=True, inferSchema=True).createOrReplaceTempView('Athletes')
spark.sql('select * from athletes order by NOC, Discipline').show(20)

+--------------------+-----------+-------------------+
|                Name|        NOC|         Discipline|
+--------------------+-----------+-------------------+
|       YOUSOFI Kimia|Afghanistan|          Athletics|
|NOOR ZAHI Sha Mah...|Afghanistan|          Athletics|
|        YOVARI Mahdi|Afghanistan|           Shooting|
|        ANWARI Fahim|Afghanistan|           Swimming|
|     MANSOURI Farzad|Afghanistan|          Taekwondo|
|       PETROV Matvei|    Albania|Artistic Gymnastics|
|       SMAJLAJ Izmir|    Albania|          Athletics|
|      CULLHAJ Indrit|    Albania|               Judo|
|     DELILAJ Manuela|    Albania|           Shooting|
|         KADIU Kledi|    Albania|           Swimming|
|       MERIZAJ Nikol|    Albania|           Swimming|
|        CALJA Briken|    Albania|      Weightlifting|
|      QERIMAJ Erkand|    Albania|      Weightlifting|
|      HETHAT Yassine|    Algeria|          Athletics|
|TRIKI Yasser Mohamed|    Algeria|          Athletics|
| LAHOULOU

In [61]:
spark.sql('''select first(NOC) as NOC, first(Discipline) as Discipline,
             count(*) as Count from athletes group by NOC, Discipline
             order by NOC, Discipline''').show(20)

+-----------+-------------------+-----+
|        NOC|         Discipline|Count|
+-----------+-------------------+-----+
|Afghanistan|          Athletics|    2|
|Afghanistan|           Shooting|    1|
|Afghanistan|           Swimming|    1|
|Afghanistan|          Taekwondo|    1|
|    Albania|Artistic Gymnastics|    1|
|    Albania|          Athletics|    1|
|    Albania|               Judo|    1|
|    Albania|           Shooting|    1|
|    Albania|           Swimming|    2|
|    Albania|      Weightlifting|    2|
|    Algeria|          Athletics|    8|
|    Algeria|             Boxing|    8|
|    Algeria|       Canoe Sprint|    1|
|    Algeria|       Cycling Road|    2|
|    Algeria|            Fencing|    3|
|    Algeria|               Judo|    2|
|    Algeria|  Marathon Swimming|    1|
|    Algeria|             Rowing|    2|
|    Algeria|            Sailing|    2|
|    Algeria|           Shooting|    1|
+-----------+-------------------+-----+
only showing top 20 rows



In [62]:
pd_athletes_query = spark.sql('''select first(NOC) as NOC, first(Discipline) as Discipline,
             count(*) as Count from athletes group by NOC, Discipline
             order by NOC, Discipline''').toPandas()

In [63]:
fig = px.bar(pd_athletes_query, x='NOC', y='Count', color='Discipline', title='Number of Athletes  from each country per discipline')
fig.show()

In [64]:
athletes['Discipline'].value_counts().head(10)

Athletics       2068
Swimming         743
Football         567
Rowing           496
Hockey           406
Judo             373
Handball         343
Shooting         342
Sailing          336
Rugby Sevens     283
Name: Discipline, dtype: int64

In [65]:
pd_athletes_query = pd_athletes_query.loc[pd_athletes_query['Discipline'].isin(['Athletics', 'Swimming', 'Football'])]
pd_athletes_query.Discipline.value_counts()

Athletics    195
Swimming     181
Football      24
Name: Discipline, dtype: int64

In [66]:
fig = px.bar(pd_athletes_query, x='NOC', y='Count', color='Discipline', title='Number of Athletes  from each country per discipline')
fig.show()

In [67]:
entriesGender.head(5)

,Discipline,Female,Male,Total
0,3x3 Basketball,32,32,64
1,Archery,64,64,128
2,Artistic Gymnastics,98,98,196
3,Artistic Swimming,105,0,105
4,Athletics,969,1072,2041


In [68]:
len(entriesGender)

46

In [69]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

specs = [[{'type':'domain'}, {'type':'domain'}]] * 23
fig = make_subplots(rows=23, cols=2, 
                    subplot_titles=entriesGender['Discipline'], specs=specs)
sub_figs= []
for index, row in entriesGender.iterrows():
  sub_fig = go.Pie(labels=['Female', 'Male'], values=[row['Female'],
                                                      row['Male']])
  sub_figs.append(sub_fig)

k = 0
for i in range(1,24):
  for j in range(1,3):
    fig.add_trace(sub_figs[k], i , j)
    k += 1

fig.update_layout(showlegend=False, height=9000, width=600, title_text="Distribution of gender amongst discipline")
fig.update_traces(textposition='inside', textinfo='label+percent',hoverinfo='label+value+percent')

fig = go.Figure(fig)
fig.show()